# Unigram Model 
---

In [49]:
from nltk.corpus import stopwords as sw
from nltk.tokenize import word_tokenize as wt
from nltk import PorterStemmer  
import re
from nltk import FreqDist
from nltk.probability import LidstoneProbDist, WittenBellProbDist
import sys
from IPython.display import display
#sys.stdout = stdout
#reload(sys)
#sys.setdefaultencoding("utf-8")
stopWord = set(sw.words("english"));
import numpy as np
import pandas as pds
import math
import os
from __future__ import print_function

# variables
---

In [50]:
files = ["ray_913742943690653739_cf.txt", "ray_925987435331224445_cf.txt", 
        "ray_925987537478928034_pf.txt"]
lam = 0.3
topNRDoc = 10;

# [Functions]

# function to clean document 
---

In [51]:
# getting word from documents

In [52]:
#cleaning the words

In [53]:
dots = ['.', ',', '!', '', "+", "#", "(", ")", ":", "'s"]
slangs = {"n't":"not", "r": "are", "u": "you"}
def goclean(doc):
    doc = doc.lower()
    #\W+
    tokens = wt(doc)
    filterWord = [];
    
    for w in tokens:
        if w not in dots and w not in stopWord:
            if w in slangs:
                w = slangs[w];
            filterWord.append(w)
    sents = " ".join(filterWord)
    filterWord = re.findall('\w+', sents)
    ps = PorterStemmer()
    stemed = []
    for w in filterWord:
        fword = ps.stem(w)
        stemed.append(fword) 
    return stemed

# function to calculate probability
---

In [54]:
def getprobability(cleanW):
    myarray = np.asarray(cleanW)
    ue, ce = np.unique(myarray, return_counts=True)
    tc = len(cleanW);
    n = float(tc)
    
    p = [];
    for i in range(len(ce)): 
        p.append(ce[i]/n)
    
    mydict = dict(zip(ue, p))
    return mydict

In [55]:
#smoothing function not in use now

In [56]:
def smooth(probd, probc, lam):
    dict2 = dict(probc)
    
    for key, items in probc.iteritems():
        if key in probd:
            val = (1 - lam)*probd[key] + lam*probc[key];
            #pdf[name][key] = val;
            dict2[key] = val
        
        else:
            val = lam*(probc[key]);
            #pdf[name][key] = val;
            dict2[key] = val
    return dict2;

In [57]:
#change in formula instead of smoothing we get prob ratio

In [58]:
def getpratio(probd, probc, lam):
    dict2 = dict(probc)
    
    for key, items in probc.iteritems():
        if key in probd:
            val = ((1 - lam)*probd[key])/(lam*probc[key]) + 1;
            #pdf[name][key] = val;
            dict2[key] = val
        
        else:
            val = 1
            #pdf[name][key] = val;
            dict2[key] = val
    return dict2;

# inverse indexing
---

In [59]:
# mapping words to document

In [60]:
def getWordMap(clrev, wordMap, i):
    for key in clrev:
        wordMap[key].append(i)
    return wordMap

In [61]:
#merging wordMap for a query to get the list of doc which contain the words

In [62]:
def getMerge(wordMap, word):
    docList = []
    for w in word:
        if w not in pdf.index:
            continue;
        docList= list(set(docList + wordMap[w]))
    return docList

# KL divergence / Ranking function
---

In [63]:
def KLdivergence(q, df, docList):
    TD = len(docList)
    x = [0]*TD;
    for word, prob in q.iteritems():
        if word not in pdf.index:
            continue;
        for i in range(TD):
            name = 'probd' + str(docList[i])
            x[i] = x[i] + prob*(math.log(pdf[name][word]))
    return x;       

# working on collection
---

In [64]:
doc = ""
def getdoc(files, doc):
    for f in files:
        doc += open(f , 'r').read()
    return doc

doc += getdoc(files, doc);
doc = doc.decode('utf-8');
cw = goclean(doc)

In [65]:
probc = getprobability(cw)
dict2 = dict(probc)
pdf = pds.DataFrame(dict2.values(), index=dict2.keys(), columns=["probc"])

In [66]:
tc = len(probc)
values = [[] for _ in range(tc)]
keys = dict2.keys()
wordMap = dict(zip(keys, values)) 

# Working on documents / Doc model
---

In [67]:
reviews = doc.split("\n")
td = len(reviews)
for i in range(len(reviews)):
    name = "probd" + str(i);
    
    clrev = goclean(reviews[i]) # ['what', 'should', 'not', 'be', 'done'] without removing stopword
    
    wordMap = getWordMap(clrev, wordMap, i) #{u'andra': [1084, 1084], u'authorit': [254, 255, 784, 254, 255, 784],..}
    
    prob = getprobability(clrev) #{'be': 0.20000000000000001,'done': 0.20000000000000001,........}
    
    ps = getpratio(prob, probc, lam) #((1 - lam)*probd[key])/(lam*probc[key]) + 1;
    
    pdf[name] = ps.values() #including the doc model in data frame

# query modeling
---

In [68]:
#q = (reviews[16] + '.')[:-1]
q = raw_input() # "what shouldn't be done"

cq = goclean(q) # ['what', 'should', 'not', 'be', 'done'] without removing stopword

qprob = getprobability(cq) #{'be': 0.20000000000000001,'done': 0.20000000000000001,........}  

docList = getMerge(wordMap, cq) #[1195, 4, 519, 522, 524, .......] list containg words in query

div = KLdivergence(qprob, pdf, docList) #ranking function using KL divergens

index = div.index(max(div)) #most probabel relevent doc

#reviews[docList[index]]

Amazon Netflix


# top N Relevent doc
---

In [69]:
tlist = div[:];
x = []#*topNRDoc

for i in range(min(topNRDoc, len(tlist))):
    Pmax = max(tlist)
    #if least == 0:
        #break
    index = tlist.index(Pmax);
    x.append(docList[index])
    tlist[index] = -1;

    
rdl = len(x)
if rdl == 0:
    print("No relevent document found")
    
else:
    prt = ""
    for i in range(rdl):
        display(str(i+1) + ') ' + reviews[x[i]])

u'1) I really liked how you related technical references back to your daily life and how you use amazon services and that of our customers, Alexa, Netflix.'

u'2)  great overview of ML experience at Amazon and other customers (Zillow, pinterest, netflix, well structured to follow you through the three-tier AI/ML model like the separation of framework in different categories and examples'

u'3)  great overview of ML experience at Amazon love the statement: we want to share the experience from other customers and ourselves with you good highlighting freedom of choice good customer examples (netflix) nice wrap-up (history, 3-tier model, breadth & depth of platform)'

u'4) deep use of amazon use cases to showcase our AI/ML capabilities; strong use of references (e.g., Netflix, AHA, etc.); very complete use of the use cases; deep review of each level of AI/ML; nice tie backs to our infrastructure (P2, etc.);'

u'5) Strong intro with summary of internal Amazon use cases over time. Like how you presented the choice we offer and support for major frameworks. Good job with overview of the 3 layers, and like how you weaved in customer reference examples like Netflix. Overall solid job establishing our leadership.'

u"6) Really good job learning and working in the customer cases (Zillow, Netflix, CSPAN, American Heart Association). Nailed the core message of three main points of 1) Amazon ML heritage, 2) the three tier AWS ML platform and 3) the three main benefits to ML from the overall AWS cloud platform of a) S3 for scalable storage tier b) high security and c) the most robust analytics offering. Also great job calling out AWS' differentiation around taking an open approach to ML/DL frameworks that other vendors are not doing."

u'7) Great use of customer examples Netflix, Expedia and Bandlab Very calm and measured delivery'

u'8) Nice job weaving in Zillow and Netflix. Great job including analytics and data lake concepts'

u'9) Super smooth delivery. Great understanding of the stack layers. Loved how you described NetFlix use case.'

u"10) Nailed the three core components of the message of 1) Amazon's strong 20 year heritage in AI-ML 2) the AWS three tier ML paltform and 3) the three main benefits of the overall AWS cloud for ML of a) S3 as a scalable and highly cost effective data lake b) AWS is a highly secure environment and c) we have the broadest analytics tools portfolio beyond ML tools. Great detail on each of the three layers of our ML platform and especially on our open support of the various ML/DL frameworks which is a key AWS differentiator. Awesome calling out the Zillow, Netflix and Liberty Mutual customer examples."